# Fitting Hyperbolas based in locations found with YOLO Model

In [1]:
#import libraries
import numpy as np
from ultralytics import YOLO
import os

#Setting Working directory
import sys
from pathlib import Path

sys.path.append(str(Path.cwd().parent))

#Import the config file so that only the Filename needs to be changed in the _read_segy function
from config import *

In [2]:
from Pipeline.Datatoolkit import DatatoolKit


dk = DatatoolKit(TEST_FILE_DIR , "UG3DQUERUNTERZUG.SGY")
file = dk.LoadSGY()
df = dk.create_df(file)

In [3]:
from Pipeline.Predictor import Predictor

p = Predictor(YOLO_MODEL_DIR, PREDICT_DIR)

In [15]:
predictions = p.predict_location_hyperbolas(conf=0.1,save=True)


image 1/3 c:\pythonad\PAINDHS25\PAINDGPR\Data\images_for_prediction\UG3DQUERUNTERZUG.SGY_inline_52.png: 384x640 5 hyperbolas, 143.9ms
image 2/3 c:\pythonad\PAINDHS25\PAINDGPR\Data\images_for_prediction\UG3DQUERUNTERZUG.SGY_inline_58.png: 384x640 6 hyperbolas, 96.5ms
image 3/3 c:\pythonad\PAINDHS25\PAINDGPR\Data\images_for_prediction\UG3DQUERUNTERZUG.SGY_inline_59.png: 384x640 9 hyperbolas, 147.6ms
Speed: 3.1ms preprocess, 129.3ms inference, 3.5ms postprocess per image at shape (1, 3, 384, 640)
Results saved to C:\pythonad\PAINDHS25\PAINDGPR\Notebooks\runs\detect\predict


In [8]:
boxes = p.extract_boxes(predictions)

In [9]:
print(boxes)

{'UG3DQUERUNTERZUG.SGY_inline_52.png': array([[      1.137,      107.46,      117.34,      161.32],
       [     6.4071,      71.518,      210.26,      127.78],
       [     1.1274,      56.227,       93.03,      110.63],
       [      1.466,      56.229,      140.63,      111.66],
       [     346.46,      7.3693,      393.12,      54.333]], dtype=float32), 'UG3DQUERUNTERZUG.SGY_inline_58.png': array([[     611.76,      8.7165,      666.77,      53.012],
       [     1.3459,      105.51,      117.38,      162.08],
       [     561.99,      9.6865,      608.43,      53.505],
       [     624.89,      48.231,      705.55,        96.4],
       [     695.53,      54.243,      710.76,      95.824],
       [     2.5244,      62.416,       159.9,      109.81]], dtype=float32), 'UG3DQUERUNTERZUG.SGY_inline_59.png': array([[    0.16984,      11.065,      32.777,      55.192],
       [     508.71,      5.6568,       557.3,      50.618],
       [     561.52,      8.3862,      609.23,      50.604

In [10]:
#translate bboxes coordinates to sgy date

def bbox_to_data(df, bbox:dict,sample_interval=None):

    results = {}

    for img_name, box_list in bbox.items():
        if "_inline_" not in img_name:
            continue
        inline_nr = int(img_name.split("_inline_")[1].split(".")[0])
        results[img_name] = []

        for bbox in box_list:
            x1,y1,x2,y2 = map(int, bbox)

            sub = df[(df["inline"] == inline_nr) & (df["crossline"].between(x1,x2))]
            sub = sub.sort_values("crossline")
            if sub.empty:
                continue

            amp_slice = np.vstack([
                np.array(row["Amplitude"])[y1:y2] for _, row in sub.iterrows()
            ]).T

            
            if sample_interval is not None:
                time_start_us = y1 * sample_interval
                time_end_us = y2 * sample_interval
            else:
                time_start_us = time_end_us = None

            results[img_name].append({
                "bbox": bbox,
                "inline": inline_nr,
                "crossline_min": int(sub["crossline"].min()),
                "crossline_max": int(sub["crossline"].max()),
                "sample_start": y1,
                "sample_end": y2,
                "time_start_us": time_start_us,
                "time_end_us": time_end_us,
                "amplitudes": amp_slice
            })

    return results


In [11]:
results = bbox_to_data(df, boxes)

In [12]:
from pprint import pprint
pprint(results)

{'UG3DQUERUNTERZUG.SGY_inline_52.png': [{'amplitudes': array([[  2242,   1111,   -485, ...,  -2121,  -4121,  -5373],
       [  3370,   2350,    166, ...,    894,  -2184,  -4597],
       [  1344,    363,  -1642, ...,   4031,   1792,   -341],
       ...,
       [ -8469, -10016,  -9353, ...,   4640,   6260,   6923],
       [-11652, -12566, -10357, ...,   4189,   4493,   4493],
       [-13923, -13128,  -9309, ...,   2626,   2228,   2387]], shape=(54, 117), dtype=int16),
                                         'bbox': array([      1.137,      107.46,      117.34,      161.32], dtype=float32),
                                         'crossline_max': 117,
                                         'crossline_min': 1,
                                         'inline': 52,
                                         'sample_end': 161,
                                         'sample_start': 107,
                                         'time_end_us': None,
                                         

## Edge Detection on bounding boxes

https://www.ultralytics.com/blog/edge-detection-in-image-processing-explained

https://blog.roboflow.com/edge-detection/



#### Canny Edge Detection

In [13]:
import cv2
import numpy as np

sigma = 1
low = 0.3
high = 0.7

edge_results = {}

for img_name , entries in results.items():
    edge_results[img_name] = []

    for e in entries:
        amp = e["amplitudes"].astype(float)

        #normalize
        img = (amp - np.min(amp)) / (np.max(amp) - np.min(amp) + 1e-9)
        img =  (img * 255).astype(np.uint8)

        blurred = cv2.GaussianBlur(img, (3,3), sigma)
        edges = cv2.Canny(blurred, int(low *255), int(high*255))

        e_new = e.copy()
        e_new["edges"] = edges
        edge_results[img_name].append(e_new)


In [14]:
import matplotlib.pyplot as plt

outputdir = EDGE_DETECT_DIR



for img_name, entries in edge_results.items():
    base = os.path.splitext(img_name)[0]
    for idx, e in enumerate(entries):
        if "edges" not in e:
            continue

        edges = e["edges"]
        if edges.dtype != np.uint8:
            edges = np.clip(edges, 0, 255).astype(np.uint8)

        out_path = os.path.join(outputdir, f"{base}_bbox{idx+1}_edges.png")
        cv2.imwrite(out_path, edges)
        